# REGLAS DE ASOCIACIÓN

##### Autor: 
* Javier Tomás Fernández Martín

## 0. Preliminares

Antes de empezar con el código, importamos todas las librerias que vamos a necesitar.

In [115]:
import pandas as pd
import numpy as np
from apyori import apriori

from sklearn.model_selection import train_test_split

Además de definir una semilla para asegurar que sea reproducible

In [116]:
random_state = 42


Y definimos un par de funciones que usaremos para evaluar módelos con distintos parámetros y compararlos entre sí

## 1.Carga de datos

In [117]:
df = pd.read_csv('bbdd.csv', index_col=0)


C:\Users\Javi\AppData\Local\Temp\ipykernel_328\1186484719.py:1: DtypeWarning: Columns (3,4,39,40,42,43,44,92,112,150,151,152,242,243,257,258,259,266,267,269,270,271,390,391) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('bbdd.csv', index_col=0)


Mostramos una pequeña muestra para asegurar que se ha cargado bien y de paso echarle un vistazo a nuestra BBDD

In [118]:
df.sample(5, random_state=random_state)

,P1_TFT8_Ashe_Tier,P1_TFT8_Ashe_Obj1,P1_TFT8_Ashe_Obj2,P1_TFT8_Ashe_Obj3,P1_TFT8_Blitzcrank_Tier,P1_TFT8_Blitzcrank_Obj1,P1_TFT8_Blitzcrank_Obj2,P1_TFT8_Blitzcrank_Obj3,P1_TFT8_Galio_Tier,P1_TFT8_Galio_Obj1,...,P2_TFT8_Syndra_Obj2,P2_TFT8_Syndra_Obj3,P2_TFT8_Urgot_Tier,P2_TFT8_Urgot_Obj1,P2_TFT8_Urgot_Obj2,P2_TFT8_Urgot_Obj3,P2_Augment1,P2_Augment2,P2_Augment3,P1_Win
3883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_SecondWind1,TFT6_Augment_TradeSectorPlus,TFT8_Augment_KaisaCarry,1
2439,0,0,0,0,3,TFT_Item_BrambleVest,TFT_Item_IonicSpark,TFT8_Item_GenAEEmblemItem,0,0,...,TFT_Item_StatikkShiv,0,0,0,0,0,TFT8_Augment_HeartTrait,TFT8_Augment_AnnieSupport,TFT8_Augment_GenAEEmblem,0
1786,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_SecondWind2,TFT6_Augment_Electrocharge2,TFT8_Augment_NunuSupport,1
2329,0,0,0,0,2,0,0,0,0,0,...,0,0,2,0,0,0,TFT6_Augment_ClearMind,TFT6_Augment_PortableForge,TFT8_Augment_ApheliosCarry,1
2567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_GrandGambler,TFT7_Augment_Preparation2,TFT8_Augment_SonaExile,0


In [119]:

cat_cols = [col for col in df.columns if 'Obj' in col or 'Augment' in col]
df[cat_cols] = df[cat_cols].astype(str)

num_cols = [col for col in df.columns if col not in cat_cols + ['P1_Win']]
cat_cols_idx = [df.columns.get_loc(col) for col in cat_cols]


In [120]:
df_train,df_test = train_test_split(df, test_size=0.2, random_state=random_state)

## 2. Generación de modelos

In [123]:
# Transacciones
transactions = []
for i in range(len(df_train)):
    transaction = []
    for j in range(len(df_train.columns)):
        column_name = df_train.columns[j]
        if "_Tier" in column_name:
            champion_name = column_name.split("_Tier")[0]
            champion_tier = df_train.iloc[i][j]
            if champion_tier > 0:
                transaction.append(f"{champion_name}_Tier {champion_tier}")
        elif "_Obj" in column_name:
            item_name = column_name.split("_Obj")[0]
            item_value = df_train.iloc[i][j]
            if item_value != "0":
                transaction.append(f"{item_name} obj{column_name[-1]} {item_value}")
        elif "Augment" in column_name:
            augment_name = column_name.replace("_", "")
            augment_value = df_train.iloc[i][j]
            if augment_value != "0":
                transaction.append(f"{augment_name} {augment_value}")
    
    
    objective_value = str(df_train.iloc[i][-1])
    if objective_value == "0":
            transaction.append("P1_Lose")
    else:
            transaction.append("P1_Win")
            
    transactions.append(transaction)


In [124]:
# Generar las reglas de asociación
results = list(apriori(transactions, min_support=0.01, min_confidence=0.7, min_lift=3, min_length=2, max_length=1))



In [125]:
# Imprimir las reglas de asociación
for r in results:
    print("{} -> {}: support={}, confidence={}, lift={}".format(
        list(r.ordered_statistics[0].items_base), list(r.ordered_statistics[0].items_add),
        r.support, r.ordered_statistics[0].confidence, r.ordered_statistics[0].lift))

['P1Augment1 TFT8_Augment_AnnieSupport'] -> ['P1_TFT8_Annie_Tier 2']: support=0.011436950146627566, confidence=0.8297872340425532, lift=3.9082520277418595
['P1Augment1 TFT8_Augment_EzrealSupport'] -> ['P1_TFT8_Ezreal_Tier 2']: support=0.011730205278592375, confidence=0.7999999999999999, lift=11.414225941422593
['P1Augment1 TFT8_Augment_GenAEEmblem'] -> ['P1_TFT8_Annie_Tier 2']: support=0.017302052785923755, confidence=0.8082191780821918, lift=3.806667675773859
['P1Augment1 TFT8_Augment_GenAEEmblem'] -> ['P1_TFT8_Zoe_Tier 2']: support=0.016422287390029325, confidence=0.7671232876712328, lift=4.381725981505702
['P1Augment1 TFT8_Augment_LeeSinCarry'] -> ['P1_TFT8_Soraka_Tier 2']: support=0.010557184750733138, confidence=0.7826086956521738, lift=4.6984078383343535
['P1Augment1 TFT8_Augment_LuluCarry'] -> ['P1_TFT8_Lulu_Tier 3']: support=0.010263929618768328, confidence=1.0, lift=43.71794871794872
['P1Augment1 TFT8_Augment_ViSupport'] -> ['P1_TFT8_Vi_Tier 2']: support=0.02434017595307918, c

In [135]:
final_rules=[]

for rule in results:
    if 'P1_Win' in rule.ordered_statistics[0].items_add:
        final_rules.append(rule)
    elif 'P1_Lose' in rule.ordered_statistics[0].items_add:
        final_rules.append(rule)

In [136]:
# Imprimir las reglas de asociación
for r in final_rules:
    print("{} -> {}: support={}, confidence={}, lift={}".format(
        list(r.ordered_statistics[0].items_base), list(r.ordered_statistics[0].items_add),
        r.support, r.ordered_statistics[0].confidence, r.ordered_statistics[0].lift))

['P1_TFT8_Alistar_Tier 2', 'P2_TFT8_Aphelios obj1 TFT_Item_LastWhisper'] -> ['P1_Lose', 'P2_TFT8_Aphelios_Tier 2']: support=0.010263929618768328, confidence=0.7291666666666666, lift=6.775090826521344
['P1_TFT8_Aphelios_Tier 2', 'P1_TFT8_Samira obj1 TFT_Item_LastWhisper'] -> ['P1_TFT8_Samira_Tier 2', 'P1_Win']: support=0.017302052785923755, confidence=0.7023809523809524, lift=7.279997105225069
['P1_TFT8_Aphelios_Tier 2', 'P1_TFT8_Samira obj2 TFT_Item_HextechGunblade'] -> ['P1_TFT8_Samira_Tier 2', 'P1_Win']: support=0.011143695014662757, confidence=0.7037037037037038, lift=7.293707080941124
['P1_TFT8_Aphelios_Tier 2', 'P1_TFT8_Samira obj3 TFT_Item_LastWhisper'] -> ['P1_TFT8_Samira_Tier 2', 'P1_Win']: support=0.013196480938416423, confidence=0.7377049180327869, lift=7.646120882953809
['P1_TFT8_Gangplank_Tier 3', 'P1_TFT8_Nilah_Tier 3'] -> ['P1_Win', 'P1_TFT8_Yuumi_Tier 3']: support=0.012316715542521995, confidence=0.7, lift=18.221374045801525
['P1_TFT8_Leona_Tier 2', 'P1_TFT8_Samira obj1 

In [137]:
final_rules

[RelationRecord(items=frozenset({'P1_TFT8_Alistar_Tier 2', 'P1_Lose', 'P2_TFT8_Aphelios obj1 TFT_Item_LastWhisper', 'P2_TFT8_Aphelios_Tier 2'}), support=0.010263929618768328, ordered_statistics=[OrderedStatistic(items_base=frozenset({'P1_TFT8_Alistar_Tier 2', 'P2_TFT8_Aphelios obj1 TFT_Item_LastWhisper'}), items_add=frozenset({'P1_Lose', 'P2_TFT8_Aphelios_Tier 2'}), confidence=0.7291666666666666, lift=6.775090826521344), OrderedStatistic(items_base=frozenset({'P1_TFT8_Alistar_Tier 2', 'P1_Lose', 'P2_TFT8_Aphelios obj1 TFT_Item_LastWhisper'}), items_add=frozenset({'P2_TFT8_Aphelios_Tier 2'}), confidence=0.875, lift=4.923679867986799)]),
 RelationRecord(items=frozenset({'P1_TFT8_Samira_Tier 2', 'P1_TFT8_Aphelios_Tier 2', 'P1_TFT8_Samira obj1 TFT_Item_LastWhisper', 'P1_Win'}), support=0.017302052785923755, ordered_statistics=[OrderedStatistic(items_base=frozenset({'P1_TFT8_Aphelios_Tier 2', 'P1_TFT8_Samira obj1 TFT_Item_LastWhisper'}), items_add=frozenset({'P1_TFT8_Samira_Tier 2', 'P1_Win

## 3. EVALUACIÓN DE MODELOS

In [138]:
def score_model(transactions, model_rules):
    """
    Calcula el score del modelo basado en reglas obtenido a partir de un dataframe y un modelo de reglas.

    :param df: Pandas DataFrame con el conjunto de datos.
    :param model_rules: Lista de reglas obtenidas a partir de Apriori.
    :return: Float con la precisión del modelo.
    """


    # Calcular las predicciones del modelo para cada transacción
    predictions = []
    for transaction in transactions:
        transaction_set = set([f"{column}={value}" for column, value in enumerate(transaction)])
        for rule in model_rules:
            if rule[0].issubset(transaction_set):
                predictions.append(1 if "P1_Win=1" in rule else 0)
                break
        else:
            # Si no se encuentra ninguna regla que se aplique a la transacción, predecir 0
            predictions.append(0)

    # Calcular la precisión del modelo
    actual_values = df["P1_Win"].values.tolist()
    correct_predictions = sum([1 if p == a else 0 for p, a in zip(predictions, actual_values)])
    accuracy = correct_predictions / len(predictions)

    return accuracy

In [139]:
score_model(transactions, final_rules)

0.5017595307917888